# Guardian - Iteration 11

## Import Packages

In [6]:
# load .env File in Environment
from dotenv import load_dotenv
# operating system interfaces - get Environment-variables
import os
# Connect to API
import requests
# create Pandas Data Frame
import pandas as pd
import datetime as dt
from functools import wraps
load_dotenv()

True

### Set Params

In [7]:
iteration = 11

content_url = "https://content.guardianapis.com/search"
all_content_params = {
    "page-size": "50",
    "show-fields": "all",
    "show-tags": "all",
    "from-date": "2017-11-01",
    "to-date": "2018-04-01",
    "order-by": "newest",
    "api-key": os.getenv("GUARDIAN_API_KEY4"),
}


In [8]:
def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"{func.__name__}:\n shape={result.shape} took {time_taken}s\n")
        return result

    return wrapper

### Create DataFrame function


In [9]:
def get_df(url: str, params: dict):
        all_results = []
        current_page = 1
        total_pages = 1
        while current_page <= total_pages:
                tic = dt.datetime.now()
                params["page"] = current_page
                try:
                        r = requests.get(url, params)
                        all_results = all_results + r.json()["response"]["results"]
                        r.raise_for_status()
                except Exception as err:
                        SystemExit(err)
                if current_page == 1:
                        print("---- API STATUS ---- ")
                        print("status",  r.json()["response"]["status"])
                        total_pages = r.json()['response']['pages']
                        print("URL: ", r.url)
                        print("status",  r.json()["response"]["status"])
                        print("total",  r.json()["response"]["total"])
                        print("startIndex",  r.json()["response"]["startIndex"])
                        print("pageSize",  r.json()["response"]["pageSize"])
                        print("pages",  r.json()["response"]["pages"])
                        print("orderBy",  r.json()["response"]["orderBy"])
                        print("---- RUNTIME STATUS ---- ")

                time_taken = str(dt.datetime.now() - tic)
                print(f"({current_page}/{total_pages}) in {time_taken}s")
                        
                current_page += 1

        return pd.DataFrame(all_results)

### API Request

In [10]:
init_df = get_df(content_url, all_content_params)

---- API STATUS ---- 
status ok
URL:  https://content.guardianapis.com/search?page-size=50&show-fields=all&show-tags=all&from-date=2017-11-01&to-date=2018-04-01&order-by=newest&api-key=295c6540-8c34-4390-ad47-0cf94ebc153f&page=1
status ok
total 33293
startIndex 1
pageSize 50
pages 666
orderBy newest
---- RUNTIME STATUS ---- 
(1/666) in 0:00:17.051836s
(2/666) in 0:00:11.405168s
(3/666) in 0:00:26.756160s
(4/666) in 0:00:34.700159s
(5/666) in 0:00:04.576697s
(6/666) in 0:00:08.029975s
(7/666) in 0:00:09.279681s
(8/666) in 0:00:13.431187s
(9/666) in 0:00:09.311656s
(10/666) in 0:00:10.645396s
(11/666) in 0:00:12.961380s
(12/666) in 0:00:06.642851s
(13/666) in 0:00:11.996386s
(14/666) in 0:00:11.954379s
(15/666) in 0:00:21.244453s
(16/666) in 0:00:10.652387s
(17/666) in 0:00:08.113267s
(18/666) in 0:00:09.708229s
(19/666) in 0:00:06.031955s
(20/666) in 0:00:30.292685s
(21/666) in 0:00:11.023420s
(22/666) in 0:00:11.019033s
(23/666) in 0:00:13.256274s
(24/666) in 0:00:08.760390s
(25/666) i

In [ ]:
@log_step
def init_pipeline(df):
    return df.copy()

@log_step
def unfold_columns(df):
    dict_cols = ["fields"] 
    for col in dict_cols:
        new_df = pd.DataFrame()
        new_df[col] = df[col]#.apply(lambda x: eval(x))
        add_cols_df = pd.json_normalize(new_df[col])
        df = pd.concat([df, add_cols_df], axis=1)
    
    # Tags extraction
    
    #df["tags"] = df["tags"].apply(lambda x: eval(x))
    df['tagWebTitle'] = df['tags'].map(lambda x:[i['webTitle'] for i in x])
    df['tagId'] = df['tags'].map(lambda x:[i['id'] for i in x])
    df = df.drop(columns="tags")
    return df.drop(columns=dict_cols)
@log_step
def drop_columns(df):
    df = df.drop(columns=[
        "body",
        "webUrl",
        "apiUrl",
        "sectionId",
        "pillarId",
        "showAffiliateLinks",
        "bylineHtml",
        "standfirst",
        "shouldHideReaderRevenue",
        "isInappropriateForSponsorship",
        "shortUrl",
        "isPremoderated",
        "charCount",
        "shouldHideAdverts",
        "showInRelatedContent",
        "lang",
        "main",
        "thumbnail"
        
    ])
    return df

@log_step
def create_csv(df):
    df.to_csv(f"../../data/partial/articles_partial_{iteration}.csv.zip", compression="zip")
    return df

In [ ]:
articles = (
    init_df
        .pipe(init_pipeline)
        .pipe(unfold_columns)
        .pipe(drop_columns)
        .pipe(create_csv)
)

init_pipeline:
 shape=(33293, 14) took 0:00:00.008534s

unfold_columns:
 shape=(33293, 17) took 0:00:00.641861s

create_csv:
 shape=(33293, 17) took 0:00:02.851827s

